In [1]:
import logging

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [3]:
dtype = tf.dtypes.float32

In [4]:
N = 10
p = 2
beta_dist = tfd.Uniform(-2, 4)
tf.random.set_seed(0)
beta = beta_dist.sample((p, 1), seed=1)
sigma = tf.constant(1.0, dtype=dtype)
print((N, p))
print(beta)
print(sigma)

(10, 2)
tf.Tensor(
[[ 1.2383811]
 [-1.3703618]], shape=(2, 1), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [5]:
X_dist = tfd.Uniform(low=-1.0, high=1.0)
error_dist = tfd.Normal(loc=0, scale=sigma)
tf.random.set_seed(0)
X = X_dist.sample((N, p), seed=0, name="X")
error = error_dist.sample((N, 1), seed=0, name="error")
y = X @ beta + error

In [6]:
beta_hat_ols = tf.linalg.inv(tf.transpose(X) @ X) @ tf.transpose(X) @ y
residual = y - X @ beta_hat_ols
sigma_hat_squared_ols = tf.matmul(residual, residual, transpose_a=True) / N
theta_hat_ols = tf.concat([beta_hat_ols[..., 0], sigma_hat_squared_ols[..., 0]], axis=0)
theta_hat_ols

<tf.Tensor: id=100, shape=(3,), dtype=float32, numpy=array([ 0.7135918, -1.060262 ,  1.4947654], dtype=float32)>

In [7]:
class Regression(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def rss(self, beta_hat):
        residuals = self.y - self.X @ beta_hat
        square_residuals = tf.transpose(residuals) @ residuals
        rss = tf.reduce_sum(square_residuals)
        return rss

    def dist(self, theta_hat):
        N, p = self.X.shape
        beta_hat = theta_hat[0:p][..., tf.newaxis]
        sigma = theta_hat[p]
        mu = self.X @ beta_hat
        return tfd.Normal(mu, sigma)

    def likelihood(self, theta_hat):
        dist = self.dist(theta_hat)
        probs = dist.prob(y)
        return tf.reduce_prod(probs)

    def neg_log_likelihood(self, theta_hat):
        dist = self.dist(theta_hat)
        log_probs = dist.log_prob(y)
        return -1 * tf.reduce_sum(log_probs)


In [8]:
regression = Regression(X, y)

In [9]:
regression.rss(beta)

<tf.Tensor: id=107, shape=(), dtype=float32, numpy=15.955604>

# Make a grid

In [16]:
betas = tf.range(start=-2.0, limit=2.0, delta=0.05)
sigmas = tf.range(start=0.5, limit=3.5, delta=0.05)

In [17]:
beta_grid, sigma_grid = tf.meshgrid(betas, sigmas)
beta_grid.shape, sigma_grid.shape
grid = tf.concat([beta_grid[..., tf.newaxis], sigma_grid[..., tf.newaxis]], axis=2)
grid.shape

TensorShape([60, 80, 2])

In [18]:
def make_rss(x1s, x2s):
    zs = np.zeros(shape=(len(x1s), len(x2s)))
    for x1_ in range(len(x1s)):
        for x2_ in range(len(x2s)):
            theta_ = tf.stack([x1s[x1_], x2s[x2_]])
            zs[x1_, x2_] = regression.rss(tf.reshape(theta_, (2, 1)))
    return tf.constant(zs)


def make_neg_log_likelihood(x1s, x2s):
    zs = np.zeros(shape=(len(x1s), len(x2s)))
    for x1_ in range(len(x1s)):
        for x2_ in range(len(x2s)):
            theta_ = tf.stack([x1s[x1_], x2s[x2_]])
            zs[x1_, x2_] = regression.neg_log_likelihood(theta_)
    return tf.constant(zs)


zs_rss = make_rss(betas, betas)
zs_nll = make_neg_log_likelihood(betas, sigmas)

<tf.Tensor: id=295383, shape=(80, 80), dtype=float64, numpy=
array([[44.05355835, 43.71123505, 43.38095856, ..., 52.89453125,
        53.47851944, 54.07453156],
       [43.10997009, 42.76990891, 42.44187546, ..., 52.12437057,
        52.71060181, 53.30886841],
       [42.18314743, 41.84533691, 41.51955414, ..., 51.37096786,
        51.95946121, 52.55997467],
       ...,
       [20.43933487, 20.27044106, 20.11357689, ..., 42.63384628,
        43.3912468 , 44.16067886],
       [20.78632355, 20.6196785 , 20.46507072, ..., 43.15425491,
        43.91391373, 44.6856041 ],
       [21.15007401, 20.98568726, 20.83332443, ..., 43.69142914,
        44.45333481, 45.22728348]])>

# Plotting!

In [19]:
import plotly.graph_objects as go

In [21]:
fig = go.Figure(data=[go.Surface(x=betas, y=betas, z=zs)])
fig.update_traces(contours_z=dict(
    show=True,
    usecolormap=True,
    highlightcolor="limegreen",
    project_z=True))


In [15]:
beta_hat_ols

<tf.Tensor: id=85, shape=(2, 1), dtype=float32, numpy=
array([[ 0.7135918],
       [-1.060262 ]], dtype=float32)>